In [7]:

#import basic modules
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import gc
import seaborn as sns

from scipy import ndimage
from subprocess import check_output

# import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler


In [3]:
train_dat=pd.read_csv('Features/train_every_features_0905.csv')
test_dat=pd.read_csv('Features/test_every_features_0905.csv')

In [101]:
label=pd.read_csv('FE and error_set/error_set_0.741.csv')
label_map = {'retained':0,'2month':1,'month':2,'week':3}

In [5]:
sess=tf.Session()

In [12]:
train_np=np.array(train_dat.iloc[:,1:])

In [99]:
test_np=np.array(test_dat.iloc[:,1:])

In [100]:
test_np

array([[2.10294084e+00, 1.63957586e+04, 3.28245110e-01, ...,
        2.00000000e+01, 2.00000000e+01, 2.00000000e+01],
       [6.99195454e-01, 2.47784792e+04, 2.36792344e-01, ...,
        6.00000000e+00, 2.00000000e+00, 3.00000000e+00],
       [0.00000000e+00, 1.76000000e+03, 8.01136364e-02, ...,
        2.00000000e+01, 2.00000000e+01, 2.00000000e+01],
       ...,
       [2.43918965e-01, 8.85540000e+03, 3.34721290e-01, ...,
        3.00000000e+00, 4.00000000e+00, 1.00000000e+00],
       [5.68261129e-01, 2.07175405e+04, 3.82929206e-01, ...,
        3.00000000e+00, 1.00000000e+00, 2.00000000e+01],
       [1.16350815e+00, 1.00102941e+03, 0.00000000e+00, ...,
        6.00000000e+00, 7.00000000e+00, 2.00000000e+00]])

In [102]:
label_np=np.array([label_map[x] for x in label.label])

In [104]:
label_np.shape

(100000,)

In [56]:
y_vals1 = np.array([1 if y=='retained' else -1 for y in label.label])
y_vals2 = np.array([1 if y=='2month' else -1 for y in label.label])
y_vals3 = np.array([1 if y=='month' else -1 for y in label.label])
y_vals4 = np.array([1 if y=='week' else -1 for y in label.label])
y_vals = np.array([y_vals1, y_vals2, y_vals3,y_vals4])

In [59]:
#training hyper parameters

learning_rate = 0.000001
C = 20
gamma = 50
batch_size=100
feature_num=np.shape(train_np)[1]

In [115]:
import numpy as np
import tensorflow as tf

def get_input_fn(dataset_split, batch_size, capacity=10000, min_after_dequeue=3000):

    def _input_fn():
        images_batch, labels_batch = tf.train.shuffle_batch(
            tensors=[dataset_split.images, dataset_split.labels.astype(np.int32)],
            batch_size=batch_size,
            capacity=capacity,
            min_after_dequeue=min_after_dequeue,
            enqueue_many=True,
            num_threads=4)
        features_map = {'images': images_batch}
        return features_map, labels_batch

    return _input_fn

data = tf.contrib.learn.datasets.mnist.load_mnist()

train_input_fn = get_input_fn(data.train, batch_size=256)
eval_input_fn = get_input_fn(data.validation, batch_size=5000)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [116]:
image_column = tf.contrib.layers.real_valued_column('images', dimension=784)
optimizer = tf.train.FtrlOptimizer(
   learning_rate=50.0, l2_regularization_strength=0.001)


kernel_mapper = tf.contrib.kernel_methods.RandomFourierFeatureMapper(
  input_dim=784, output_dim=2000, stddev=5.0, name='rffm')
kernel_mappers = {image_column: [kernel_mapper]}
estimator = tf.contrib.kernel_methods.KernelLinearClassifier(
   n_classes=10, optimizer=optimizer, kernel_mappers=kernel_mappers)

# Train.
start = time.time()
estimator.fit(input_fn=train_input_fn, steps=2000)
end = time.time()
print('Elapsed time: {} seconds'.format(end - start))

# Evaluate and report metrics.
eval_metrics = estimator.evaluate(input_fn=eval_input_fn, steps=1)
print(eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7efdc4bef0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmpbq9a34zj'}
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.EstimatorSpec. You can use the `estimator_spec` method to create an equivalent one.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running lo

ResourceExhaustedError: OOM when allocating tensor of shape [2000] and type float
	 [[Node: Const_1 = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [2000] values: 5.96000051 0.0609616861 5.7631321...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'Const_1', defined at:
  File "/opt/conda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/opt/conda/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()


In [117]:
import numpy as np
import tensorflow as tf

def get_input_fn(x_data, y_data, batch_size, capacity=10000, min_after_dequeue=3000):
    def _input_fn():
        x_batch, y_batch = tf.train.shuffle_batch(
            tensors=[x_data, y_data.astype(np.int32)],
            batch_size=batch_size,
            capacity=capacity,
            min_after_dequeue=min_after_dequeue,
            enqueue_many=True,
            num_threads=4)
        features_map = {'inputs': x_batch}
        return features_map, y_batch
    return _input_fn

X_train=train_np.astype(np.float32)
y_train=label_np.astype(np.float32)
X_val=train_np.astype(np.float32)
y_val=label_np.astype(np.float32)

train_input_fn = get_input_fn(X_train,y_train, batch_size=1000)
eval_input_fn = get_input_fn(X_val,y_val, batch_size=5000)

In [ ]:
# Specify the feature(s) to be used by the estimator. This is identical to the
# code used for the LinearClassifier.
x_data=tf.contrib.layers.real_valued_column('inputs',dimension=feature_num)
optimizer = tf.train.FtrlOptimizer(
   learning_rate=50.0, l2_regularization_strength=0.001)


kernel_mapper = tf.contrib.kernel_methods.RandomFourierFeatureMapper(
  input_dim=feature_num, output_dim=2000, stddev=5.0, name='rffm')
kernel_mappers = {x_data: [kernel_mapper]}
estimator = tf.contrib.kernel_methods.KernelLinearClassifier(
   n_classes=10, optimizer=optimizer, kernel_mappers=kernel_mappers)

# Train.
start = time.time()
estimator.fit(input_fn=train_input_fn, steps=2000)
end = time.time()
print('Elapsed time: {} seconds'.format(end - start))

# Evaluate and report metrics.
eval_metrics = estimator.evaluate(input_fn=eval_input_fn, steps=1)
print(eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7efc1ef400>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmpk1ofccl8'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
